In [1]:

import pandas as pd

df1 = pd.DataFrame(
    {
        "student_id": [1, 2, 3, 4, 5],
        "name": ["Степан", "Тимофей", "Кай", "Савелий", "Никита"],
        "major": ["Математика", "Экономика", "Физика", "Математика", "Литература"],
    }
)

df2 = pd.DataFrame(
    {
        "student_id": [1, 2, 3, 6, 7],
        "course": ["Матан", "Финансы", "Термодинамика", "Ботаника", "Статистика"],
        "grade": [5, 4, 5, 3, 4],
    }
)

print("DataFrame 1 - Студенты:")
print(df1)
print("\nDataFrame 2 - Оценки:")
print(df2)

inner_join = pd.merge(df1, df2, on="student_id", how="inner")
print("\n1) Inner join (только общие студенты в обоих таблицах):")
print(inner_join)
# Результат: только студенты с id 1, 2, 3

left_join = pd.merge(df1, df2, on="student_id", how="left")
print("\n2) Left join (все студенты из df1, даже без оценок):")
print(left_join)
# Результат: все студенты из df1 (1-5), но у студентов 4 и 5 нет grade и course

right_join = pd.merge(df1, df2, on="student_id", how="right")
print("\n3) Right join (все оценки из df2, даже для студентов не из df1):")
print(right_join)
# Результат: все оценки из df2 (студенты 1, 2, 3, 6, 7), но у 6 и 7 нет информации из df1

outer_join = pd.merge(df1, df2, on="student_id", how="outer")
print("\n4) Outer join (все записи из обеих таблиц):")
print(outer_join)
# Результат: все студенты (1-7), включая тех, кто есть только в одной таблице

# Создать два DataFrame:
# df1: студенты (student_id, name, major)
# df2: оценки (student_id, course, grade)

# TODO:
# 1) Inner join
# 2) Left join
# 3) Right join
# 4) Outer join
# 5) Объяснить разницу

DataFrame 1 - Студенты:
   student_id     name       major
0           1   Степан  Математика
1           2  Тимофей   Экономика
2           3      Кай      Физика
3           4  Савелий  Математика
4           5   Никита  Литература

DataFrame 2 - Оценки:
   student_id         course  grade
0           1          Матан      5
1           2        Финансы      4
2           3  Термодинамика      5
3           6       Ботаника      3
4           7     Статистика      4

1) Inner join (только общие студенты в обоих таблицах):
   student_id     name       major         course  grade
0           1   Степан  Математика          Матан      5
1           2  Тимофей   Экономика        Финансы      4
2           3      Кай      Физика  Термодинамика      5

2) Left join (все студенты из df1, даже без оценок):
   student_id     name       major         course  grade
0           1   Степан  Математика          Матан    5.0
1           2  Тимофей   Экономика        Финансы    4.0
2           3    

## Разница между типами объединений:

### Inner join:

    Возвращает только те строки, где есть совпадения в обоих таблицах
    Аналогично пересечению множеств в математике

### Left join:

    Возвращает все строки из левой таблицы (первой в merge)
    Добавляет данные из правой таблицы, где есть совпадения
    Для строк без совпадений в правой таблице - значения NaN

### Right join:

    Возвращает все строки из правой таблицы (второй в merge)
    Добавляет данные из левой таблицы, где есть совпадения
    Для строк без совпадений в левой таблице - значения NaN

### Outer join:

    Возвращает все строки из обеих таблиц
    Объединяет данные, где есть совпадения
    Для строк без совпадений - значения NaN в соответствующих колонках

### Простая аналогия:

    Inner: только общие друзья у двух людей
    Left: все друзья первого человека + общие друзья
    Right: все друзья второго человека + общие друзья
    Outer: все друзья обоих людей

In [2]:
import pandas as pd
import numpy as np
import time

# Загрузка датасета
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df: pd.DataFrame = pd.read_csv(url)


# 1)
def categorize_family_size(row):
    """Категоризация размера семьи"""
    if pd.isna(row['SibSp']) or pd.isna(row['Parch']):
        return 'Unknown'

    family_size = row['SibSp'] + row['Parch']

    if family_size == 0:
        return 'Alone'
    elif 1 <= family_size <= 3:
        return 'Small'
    elif 4 <= family_size <= 6:
        return 'Medium'
    else:
        return 'Large'


# Применяем функцию к каждой строке
df['Family_Category'] = df.apply(categorize_family_size, axis=1)
print(f"1) Custom feature через apply (Family_Category): \n{df['Family_Category'].value_counts()}")

# 2)
df['Title'] = df['Name'].apply(lambda x: x.split(',')[1].split('.')[0].strip())
print(f"\nИзвлечение Title через lambda: \n{df['Title'].value_counts().head()}")

# Lambda с несколькими условиями
df['Age_Category'] = df['Age'].apply(lambda x: 'Child' if x < 18 else 'Adult' if x < 60 else 'Senior')
print(f"\nКатегоризация возраста через lambda: \n{df['Age_Category'].value_counts()}")

# 3)
# Создаем тестовую колонку
df['Test_Column'] = np.random.randint(1, 100, len(df))

# Метод 1: apply с lambda (медленно)
start_time = time.time()
df['Test_Lambda'] = df['Test_Column'].apply(lambda x: x * 2 + 10)
lambda_time = time.time() - start_time

# Метод 2: Векторизованная операция (быстро)
start_time = time.time()
df['Test_Vectorized'] = df['Test_Column'] * 2 + 10
vectorized_time = time.time() - start_time


# Метод 3: apply с функцией (самый медленный)
def custom_transform(x):
    return x * 2 + 10


start_time = time.time()
df['Test_Apply_Func'] = df['Test_Column'].apply(custom_transform)
func_time = time.time() - start_time

print(f"\n3) Сравнение производительности:")
print(f"Lambda apply: {lambda_time:.5f} секунд")
print(f"Vectorized: {vectorized_time:.5f} секунд")
print(f"Apply с функцией: {func_time:.5f} секунд")
print(f"Векторизованная операция быстрее lambda в {lambda_time / vectorized_time:.1f} раз!")

# Удаляем тестовые колонки
df = df.drop(['Test_Column', 'Test_Lambda', 'Test_Vectorized', 'Test_Apply_Func'], axis=1)

# TODO:
# 1) Использовать apply для создания custom feature
# 2) Использовать lambda для простых трансформаций
# 3) Сравнить производительность: apply vs vectorized operation
# 4) Понять, когда нужен apply, а когда можно векторизовать

1) Custom feature через apply (Family_Category): 
Family_Category
Alone     537
Small     292
Medium     49
Large      13
Name: count, dtype: int64

Извлечение Title через lambda: 
Title
Mr        517
Miss      182
Mrs       125
Master     40
Dr          7
Name: count, dtype: int64

Категоризация возраста через lambda: 
Age_Category
Adult     575
Senior    203
Child     113
Name: count, dtype: int64

3) Сравнение производительности:
Lambda apply: 0.00047 секунд
Vectorized: 0.00035 секунд
Apply с функцией: 0.00045 секунд
Векторизованная операция быстрее lambda в 1.3 раз!



## Когда нужен apply, а когда можно векторизовать:

(всегда сначала пробуйте векторизованные операции - они используют оптимизации NumPy/Pandas)

### Векторизировать стоит, когда

    Простые математические операции
    Операции сравнения
    Строковые операции с векторизацией

### Использовать apply стоит, когда

    Нужна сложная логика, которая зависит от нескольких колонок
    Использование внешних функций или библиотек
    Обработка с условиями, которые сложно выразить векторизованно

In [3]:
# 1) Группировка по двум колонкам: класс и пол
grouped_2cols = (
    df.groupby(["Pclass", "Sex"])
    .agg({"Survived": "mean", "Age": "mean", "Fare": "mean"})
    .round(2)
)
print(f"\n1) Множественная группировка по Pclass и Sex: \n{grouped_2cols}")

# Группировка по трем колонкам: класс, пол и порт посадки
grouped_3cols = (
    df.groupby(["Pclass", "Sex", "Embarked"])
    .agg({"Survived": ["mean", "count"], "Age": "mean", "Fare": "mean"})
    .round(2)
)
print(
    f"\nГруппировка по трем колонкам (Pclass, Sex, Embarked): \n{grouped_3cols.head(10)}"
)

# 2) Разные агрегации для разных колонок
agg_different = (
    df.groupby("Pclass")
    .agg(
        {
            "Age": ["mean", "median", "min", "max", "std", "count"],
            "Fare": ["min", "max", "mean", "median", "sum"],
            "Survived": ["mean", "sum"],
        }
    )
    .round(2)
)
print(f"\n2) Разные агрегации для разных колонок: \n{agg_different}")

# Более читаемый вариант с переименованием колонок
agg_renamed = (
    df.groupby("Pclass")
    .agg(
        age_mean=("Age", "mean"),
        age_median=("Age", "median"),
        fare_mean=("Fare", "mean"),
        fare_sum=("Fare", "sum"),
        survival_rate=("Survived", "mean"),
        total_passengers=("PassengerId", "count"),
    )
    .round(2)
)
print(f"\nГруппировка с именованными агрегациями: \n{agg_renamed}")


# 3) Пользовательские функции для агрегации
def age_range(series):
    """Диапазон возраста в группе"""
    return series.max() - series.min()


def fare_per_age(row):
    """Отношение стоимости билета к возрасту"""
    if row["Age"] > 0:
        return row["Fare"] / row["Age"]
    return np.nan


def survival_variance(series):
    """Дисперсия выживаемости (для 0/1 переменной)"""
    mean: float = series.mean()
    return ((series - mean) ** 2).mean()


# Сначала создадим колонку Fare_per_Age
df["Fare_per_Age"] = df.apply(fare_per_age, axis=1)

# Применяем кастомные агрегации
custom_agg = (
    df.groupby(["Pclass", "Sex"])
    .agg(
        {
            "Age": [age_range, "mean", "std"],
            "Fare": ["mean", "median"],
            "Fare_per_Age": "mean",
            "Survived": [survival_variance, "mean"],
        }
    )
    .round(2)
)
print(f"\n3) Custom aggregation функции: \n{custom_agg.head()}")


# Еще пример: кастомная функция для строковых данных
def most_common_name(series):
    """Самое частое имя в группе"""
    if len(series) == 0:
        return None
    # Извлекаем имена из полных имен
    names = series.str.split(",").str[0]
    return names.mode()[0] if not names.mode().empty else None


name_by_class = df.groupby("Pclass")["Name"].agg(most_common_name)
print(f"\nСамое частое имя по классам: \n{name_by_class}")


# TODO:
# 1) Множественная группировка (по 2-3 колонкам)
# 2) Разные агрегации для разных колонок:
#    df.groupby('Pclass').agg({
#        'Age': ['mean', 'median'],
#        'Fare': ['min', 'max', 'mean']
#    })
# 3) Custom aggregation функция
# 4) Transform vs Aggregate - понять разницу


1) Множественная группировка по Pclass и Sex: 
               Survived    Age    Fare
Pclass Sex                            
1      female      0.97  34.61  106.13
       male        0.37  41.28   67.23
2      female      0.92  28.72   21.97
       male        0.16  30.74   19.74
3      female      0.50  21.75   16.12
       male        0.14  26.51   12.66

Группировка по трем колонкам (Pclass, Sex, Embarked): 
                       Survived          Age    Fare
                           mean count   mean    mean
Pclass Sex    Embarked                              
1      female C            0.98    43  36.05  115.64
              Q            1.00     1  33.00   90.00
              S            0.96    48  32.70   99.03
       male   C            0.40    42  40.11   93.54
              Q            0.00     1  44.00   90.00
              S            0.35    79  41.90   52.95
2      female C            1.00     7  19.14   25.27
              Q            1.00     2  30.00   12.35
 

## Transform vs Aggregate - ключевые различия:

### Размер результата:

    Aggregate: Возвращает сводную таблицу с количеством строк = количеству уникальных групп

    Transform: Возвращает данные того же размера, что и исходные (одна строка на каждую исходную строку)

### Назначение:

    Aggregate: Для получения статистик по группам (отчеты, сводки)

    Transform: Для создания новых колонок на основе групповых вычислений (feature engineering)

### Возвращаемое значение:

    Aggregate: Новый DataFrame с агрегированными значениями

    Transform: Series/DataFrame того же индекса, что и исходные данные

### Использование:

    Aggregate: Когда нужно "свернуть" данные по группам

    Transform: Когда нужно сохранить структуру данных, но применить групповые операции

### Аналогия:

    Aggregate - как "итог по отделам" в Excel (сводная таблица)

    Transform - как "добавить колонку с процентом от отдела" к каждой записи